In [1]:
import numpy as np 
import pandas as pd 
import scanpy as sc 
import anndata as ad 
import os 
import snapatac2 as snap 
import muon as mu

# Description 

This notebook is used to generate the gene activity data in mouse brain.

<font color ='red'>Noted that the notebook contains another 10X multi-omics data, which is 

M_Brain_Chromium_Nuc_Isolation_vs_SaltyEZ_vs_ComplexTissueDP_filtered_feature_bc_matrix.h5, </font> 


This data is a demo dataset in 10X webistes, the link is available until 2025.08.29 [dataset link](https://www.10xgenomics.com/datasets/mouse-brain-nuclei-isolated-with-chromium-nuclei-isolation-kit-saltyez-protocol-and-10x-complex-tissue-dp-ct-sorted-and-ct-unsorted-1-standard) 


This data is not used in our analysis. We include this data just due to another projects is required this public data. If you want to run this notebook directly, 
please download this data. Otherwise, you need to modify some code blocks to make it work. 

This public data does not make differences to our analysis.

## rna data prepare

In [78]:
path_dic = {
    'bq': '/home/rsun@ZHANGroup.local/sly_data/data/M/MBQ/filtered_feature_bc_matrix.h5',
    'ct': '/home/rsun@ZHANGroup.local/sly_data/data/M/MCT/filtered_feature_bc_matrix.h5',
    'hm': '/home/rsun@ZHANGroup.local/sly_data/data/M/MHM/filtered_feature_bc_matrix.h5',
    'ng': '/home/rsun@ZHANGroup.local/sly_data/data/M/MNG/filtered_feature_bc_matrix.h5',
    'pc': '/home/rsun@ZHANGroup.local/sly_data/data/M/MPC/filtered_feature_bc_matrix.h5',
    'qn': '/home/rsun@ZHANGroup.local/sly_data/data/M/MQN/filtered_feature_bc_matrix.h5',
    'xq': '/home/rsun@ZHANGroup.local/sly_data/data/M/MXQ/filtered_feature_bc_matrix.h5',
    'yb': '/home/rsun@ZHANGroup.local/sly_data/data/M/MYB/filtered_feature_bc_matrix.h5',
    '10x_multi': '/home/rsun@ZHANGroup.local/sr_project/eval_data/10X_mus_data/M_Brain_Chromium_Nuc_Isolation_vs_SaltyEZ_vs_ComplexTissueDP_filtered_feature_bc_matrix.h5'
}

data_list = []

for key in path_dic:
    path = path_dic[key]
    data = sc.read_10x_h5(path)
    data.var_names_make_unique()
    data.obs.loc[:,'tissues'] = key 
    data.obs.index = key + '_' + data.obs.index.values 
    data_list.append(data)


/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/rsun@ZHANGroup.loc

In [79]:
rna_data = ad.concat(data_list)
rna_data.var = data.var.copy()

print(rna_data.shape)
print(rna_data.obs.tissues.value_counts())

(107553, 32285)
tissues
10x_multi    23990
xq           12119
ct           11772
bq           10808
pc           10767
qn           10074
yb           10029
ng            9422
hm            8572
Name: count, dtype: int64


In [80]:
sc.pp.filter_cells(rna_data, min_genes = 200)
print(rna_data.shape)
print(rna_data.obs.tissues.value_counts())

(106694, 32285)
tissues
10x_multi    23877
xq           12055
ct           11692
bq           10764
pc           10744
qn           10063
yb            9661
ng            9383
hm            8455
Name: count, dtype: int64


In [81]:
rna_data.obs.tissues.value_counts()

tissues
10x_multi    23877
xq           12055
ct           11692
bq           10764
pc           10744
qn           10063
yb            9661
ng            9383
hm            8455
Name: count, dtype: int64

In [82]:
rna_data.write_h5ad('merge_dataset/rna.h5ad')

... storing 'tissues' as categorical
... storing 'feature_types' as categorical
... storing 'genome' as categorical
... storing 'interval' as categorical


## atac data prepare

In [8]:
output_dir = "h5ad_output_1"
os.makedirs(output_dir, exist_ok=True)

fragment_paths = {
    'bq':'/home/rsun@ZHANGroup.local/sly_data/data/M/MBQ/atac_fragments.tsv.gz',
    'ct':'/home/rsun@ZHANGroup.local/sly_data/data/M/MCT/atac_fragments.tsv.gz',
    'hm':'/home/rsun@ZHANGroup.local/sly_data/data/M/MHM/atac_fragments.tsv.gz',
    'ng':'/home/rsun@ZHANGroup.local/sly_data/data/M/MNG/atac_fragments.tsv.gz',
    'pc':'/home/rsun@ZHANGroup.local/sly_data/data/M/MPC/atac_fragments.tsv.gz',
    'qn':'/home/rsun@ZHANGroup.local/sly_data/data/M/MQN/atac_fragments.tsv.gz',
    'xq':'/home/rsun@ZHANGroup.local/sly_data/data/M/MXQ/atac_fragments.tsv.gz',
    'yb':'/home/rsun@ZHANGroup.local/sly_data/data/M/MYB/atac_fragments.tsv.gz',
    '10x_multi': '/home/rsun@ZHANGroup.local/sr_project/eval_data/10X_mus_data/M_Brain_Chromium_Nuc_Isolation_vs_SaltyEZ_vs_ComplexTissueDP_atac_fragments.tsv.gz'
}

In [9]:
%%time
outputs = []
for key in fragment_paths:
    outputs.append(f'{output_dir}/{key}.h5ad')

adatas = snap.pp.import_data([fragment_paths[key] for key in fragment_paths], 
                            file=outputs, 
                            chrom_sizes=snap.genome.mm10,
                            sorted_by_barcode = False,
                            min_num_fragments=1000)#, min_tsse=7) 2.7 版本没有 min_tsse

  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [30:14<00:00, 201.56s/it]


CPU times: user 166 ms, sys: 83.2 ms, total: 249 ms
Wall time: 30min 15s


In [12]:
%%time
tissue_key = ['bq', 'ct', 'hm', 'ng', 'pc', 'qn', 'xq', 'yb','10x_mulit']
data = snap.AnnDataSet(
    adatas=[(name, adata) for name, adata in zip(tissue_key, adatas)],
    filename="atac.h5ads"
)
data

CPU times: user 263 ms, sys: 160 ms, total: 424 ms
Wall time: 228 ms


AnnDataSet object with n_obs x n_vars = 139055 x 0 backed at 'atac.h5ads'
contains 9 AnnData objects with keys: 'bq', 'ct', 'hm', 'ng', 'pc', 'qn', 'xq', 'yb', '10x_mulit'
    obs: 'sample'
    uns: 'AnnDataSet', 'reference_sequences'

In [13]:
%%time
snap.metrics.tsse(data, snap.genome.mm10)
snap.pl.tsse(data, interactive=False)

RuntimeError: not found: n_fragment

In [15]:
unique_cell_ids = [sa + '_' + bc for sa, bc in zip(data.obs['sample'], data.obs_names)]
data.obs_names = unique_cell_ids

/tmp/ipykernel_521580/2725754147.py:1: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  unique_cell_ids = [sa + '_' + bc for sa, bc in zip(data.obs['sample'], data.obs_names)]


In [17]:
%%time
#snap.tl.macs3(data)#, replicate='sample')

2025-03-02 16:07:00 - INFO - [15994 MB] #3 Pre-compute pvalue-qvalue table...
2025-03-02 20:45:29 - INFO - [46552 MB] #3 Call peaks for each chromosome...


CPU times: user 5h 35min 48s, sys: 32min 30s, total: 6h 8min 19s
Wall time: 5h 38min 18s


### get peak data

In [29]:
snap.tl.macs3(data,  groupby='sample')

2025-03-03 01:44:34 - INFO - Exporting fragments...
2025-03-03 02:11:26 - INFO - Calling peaks...
100%|██████████| 9/9 [1:22:47<00:00, 551.92s/it] 


In [34]:
merged_peaks = snap.tl.merge_peaks(data.uns['macs3'], chrom_sizes=snap.genome.hg38)
merged_peaks.shape

/tmp/ipykernel_521580/2267821930.py:1: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  merged_peaks = snap.tl.merge_peaks(data.uns['macs3'], chrom_sizes=snap.genome.hg38)


(625407, 10)

In [60]:
chr = pd.DataFrame(data.uns['macs3_pseudobulk']['chrom'])
start = pd.DataFrame(data.uns['macs3_pseudobulk']['start']) 
end = pd.DataFrame(data.uns['macs3_pseudobulk']['end']) 

raw_peaks = []

for i in range(len(chr)):
    raw_peaks.append(chr.values[i] + ':' + str(start.values[i][0]) + '-' + str(end.values[i][0]))

raw_peaks = np.array(raw_peaks).reshape(-1)

/tmp/ipykernel_521580/3022438851.py:1: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  chr = pd.DataFrame(data.uns['macs3_pseudobulk']['chrom'])
/tmp/ipykernel_521580/3022438851.py:2: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  start = pd.DataFrame(data.uns['macs3_pseudobulk']['start'])
/tmp/ipykernel_521580/3022438851.py:3: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  end = pd.DataFrame(data.uns['macs3_pseudobulk']['end'])


In [61]:
peak_mat_raw = snap.pp.make_peak_matrix(data, use_rep = raw_peaks)
peak_mat_raw.shape

/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


(139055, 470523)

In [41]:
peak_mat = snap.pp.make_peak_matrix(data, use_rep = merged_peaks['Peaks'])
peak_mat.shape

/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


(139055, 625407)

In [62]:
peak_mat_raw.write_h5ad('merge_dataset/peak_mat_raw.h5ad')
peak_mat.write_h5ad('merge_dataset/peak_mat.h5ad')

... storing 'sample' as categorical
... storing 'sample' as categorical


### get gadata

In [63]:
gadata = snap.pp.make_gene_matrix(data, snap.genome.mm10)
print(gadata.shape)

/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


(139055, 55291)


In [64]:
gadata.write_h5ad('merge_dataset/gadata.h5ad')

... storing 'sample' as categorical


## merge data and filter low quality cells

In [125]:
gadata = sc.read_h5ad('merge_dataset/gadata.h5ad')
peak_mat = sc.read_h5ad('merge_dataset/peak_mat.h5ad')
rna = sc.read_h5ad('/home/rsun@ZHANGroup.local/sr_project/eval_data/merge_dataset/rna.h5ad')

print(gadata.shape, peak_mat.shape, rna.shape)

/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


(139055, 55291) (139055, 625407) (106694, 32285)


In [131]:
## modify the obs index, and sample, because we introduce the wrong key '10x_mulit' instead of '10x_multi'
"""
new_idx = []

for ele in gadata.obs.index:
    if '10x_mulit' in ele:
        new_idx.append(ele.replace('10x_mulit', '10x_multi'))
    else:
        new_idx.append(ele)

gadata.obs.index = new_idx 


new_idx = []

for ele in peak_mat.obs.index:
    if '10x_mulit' in ele:
        new_idx.append(ele.replace('10x_mulit', '10x_multi'))
    else:
        new_idx.append(ele)

peak_mat.obs.index = new_idx 
"""

"\nnew_idx = []\n\nfor ele in gadata.obs.index:\n    if '10x_mulit' in ele:\n        new_idx.append(ele.replace('10x_mulit', '10x_multi'))\n    else:\n        new_idx.append(ele)\n\ngadata.obs.index = new_idx \n\n\nnew_idx = []\n\nfor ele in peak_mat.obs.index:\n    if '10x_mulit' in ele:\n        new_idx.append(ele.replace('10x_mulit', '10x_multi'))\n    else:\n        new_idx.append(ele)\n\npeak_mat.obs.index = new_idx \n"

In [127]:
"""
new_sample = []

for ele in gadata.obs['sample']:
    if '10x_mulit' in ele:
        new_sample.append(ele.replace('10x_mulit', '10x_multi'))
    else:
        new_sample.append(ele)

gadata.obs['sample'] = new_sample 

new_sample = []

for ele in peak_mat.obs['sample']:
    if '10x_mulit' in ele:
        new_sample.append(ele.replace('10x_mulit', '10x_multi'))
    else:
        new_sample.append(ele)

peak_mat.obs['sample'] = new_sample 
"""

In [128]:
#gadata.write_h5ad('merge_dataset/gadata.h5ad')
#peak_mat.write_h5ad('merge_dataset/peak_mat.h5ad')

... storing 'sample' as categorical
... storing 'sample' as categorical


In [132]:
print(gadata.shape)
sc.pp.filter_cells(gadata, min_genes= 200)
print(gadata.shape)

(139055, 55291)
(139043, 55291)


In [133]:
print(peak_mat.shape)
low_tsse = (peak_mat.obs.tsse >= 7)
peak_mat = peak_mat[low_tsse] 
print(peak_mat.shape)

sc.pp.filter_cells(peak_mat, min_genes=2000)
print(peak_mat.shape)
sc.pp.filter_genes(peak_mat, min_cells=20)
print(peak_mat.shape)

(139055, 625407)
(94066, 625407)


/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:168: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_genes"] = number
/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/rsun@ZHANGro

(82621, 625407)
(82621, 608869)


In [134]:
mutual_idx = np.intersect1d(rna.obs.index.values, peak_mat.obs.index.values)
print(mutual_idx.shape)
mutual_idx = np.intersect1d(mutual_idx, gadata.obs.index.values)
print(mutual_idx.shape)

(78886,)
(78886,)


In [135]:
rna = rna[mutual_idx]
peak_mat = peak_mat[mutual_idx]
gadata = gadata[mutual_idx] 

mdata = mu.MuData({'rna_count': rna, 'peak_count': peak_mat, 'ga_count': gadata})
mdata

/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/mudata/_core/mudata.py:931: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(
/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. U

MuData object with n_obs × n_vars = 78886 × 696445
  3 modalities
    rna_count:	78886 x 32285
      obs:	'tissues', 'n_genes'
      var:	'gene_ids', 'feature_types', 'genome', 'interval'
    peak_count:	78886 x 608869
      obs:	'sample', 'tsse', 'n_genes'
      var:	'n_cells'
    ga_count:	78886 x 55291
      obs:	'sample', 'tsse', 'n_genes'

In [136]:
mdata.write_h5mu('merge_dataset/mdata.h5mu')

/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/rsun@ZHANGroup.local/anaconda3/envs/snapatac/lib/python3.10/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


## generate the split index

In [142]:
from sklearn.model_selection import train_test_split

obs_info = mdata['rna_count'].obs

save_dir = 'data_split'
os.makedirs(save_dir, exist_ok=True)

In [143]:
## split 1: 10x only 
total_id = np.arange(obs_info.shape[0])

tenx_id = total_id[obs_info.loc[:,'tissues'] == '10x_multi']
train_id, test_id = train_test_split(tenx_id, test_size = 0.1, random_state = 42)
print(train_id.shape, test_id.shape)

np.save(os.path.join(save_dir, 'train_id_1.npy'), train_id)
np.save(os.path.join(save_dir, 'test_id_1.npy'), test_id) 

(10101,) (1123,)


In [144]:
## split 2: self only 
total_id = np.arange(obs_info.shape[0])

tenx_id = total_id[obs_info.loc[:,'tissues'] != '10x_multi']
train_id, test_id = train_test_split(tenx_id, test_size = 0.1, random_state = 42)
print(train_id.shape, test_id.shape)

np.save(os.path.join(save_dir, 'train_id_2.npy'), train_id)
np.save(os.path.join(save_dir, 'test_id_2.npy'), test_id) 

(60895,) (6767,)


In [145]:
## split 3: uniform split
total_id = np.arange(obs_info.shape[0])

#tenx_id = total_id[obs_info.loc[:,'tissues'] != '10x_multi']
train_id, test_id = train_test_split(total_id, test_size = 0.1, random_state = 42)
print(train_id.shape, test_id.shape)

np.save(os.path.join(save_dir, 'train_id_3.npy'), train_id)
np.save(os.path.join(save_dir, 'test_id_3.npy'), test_id) 

(70997,) (7889,)


In [146]:
## split 4: self train, 10x test
total_id = np.arange(obs_info.shape[0])

train_id = total_id[obs_info.loc[:,'tissues'] != '10x_multi']
test_id = total_id[obs_info.loc[:,'tissues'] == '10x_multi']
print(train_id.shape, test_id.shape)

np.save(os.path.join(save_dir, 'train_id_4.npy'), train_id)
np.save(os.path.join(save_dir, 'test_id_4.npy'), test_id) 

(67662,) (11224,)
